In [1]:
import os
import sys

sys.path.insert(0, '/kaggle/input/linear_mlp/pytorch/default/6')

In [2]:
!pip install --no-index --find-links=/kaggle/input/library/pytorch/default/3 pqdm

Looking in links: /kaggle/input/library/pytorch/default/3


In [3]:
!pip install --no-index --find-links=/kaggle/input/library/pytorch/default/3 astropy

Looking in links: /kaggle/input/library/pytorch/default/3


In [4]:
from utility import predict_func_2input_2out, load_best_model, cal_flux, predict_spectra, postprocessing
from smalltransformer_w_sigma import SmallTransformer
from linear_process import preprocess_pro
import pandas as pd
import numpy as np
import torch
import pickle

In [5]:
predictions_spectra_test = preprocess_pro('test')

QUEUEING TASKS | :   0%|          | 0/673 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/673 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/673 [00:00<?, ?it/s]

QUEUEING TASKS | :   0%|          | 0/673 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/673 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/673 [00:00<?, ?it/s]

In [6]:
# Model
batch_size = 1
model = SmallTransformer().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
# model, optimizer, best_val_loss, target_min, target_max = load_best_model(
#     model, optimizer,
#     '/kaggle/input/noisereduction/pytorch/default/1/best_model_NR_v2.0.pth')
model, optimizer, best_val_loss, target_min, target_max = load_best_model(
    model, optimizer, '/kaggle/best_model_STws_v3.4.pth')

Best model loaded.


In [7]:
# 特征
full_whitelight_s_test = np.array([
    predict_spectra(predictions_spectra_test[i])
    for i in range(len(predictions_spectra_test))
])  # 预测每个星球的白光缩放比例S
full_light_alpha_test = np.array([
    cal_flux(predictions_spectra_test[i])
    for i in range(len(predictions_spectra_test))
])  # 计算每个星球的各个波段的吸收峰相对面积

In [8]:
min_values = predictions_spectra_test.min(axis=(1, 2), keepdims=True)
max_values = predictions_spectra_test.max(axis=(1, 2), keepdims=True)
normalized_spectra = (predictions_spectra_test - min_values) / (max_values -
                                                                min_values)
data_test_reshaped = torch.tensor(predictions_spectra_test).float()

In [9]:
whitelight_s_test = full_whitelight_s_test
light_alpha_test = full_light_alpha_test
whitelight_s_test_expanded = np.expand_dims(full_whitelight_s_test * 100,
                                            axis=1)
combined_array = np.concatenate((light_alpha_test, whitelight_s_test_expanded),
                                axis=1)
light_alpha_test = torch.tensor(combined_array).float()

In [10]:
mean_predictions, sigma_predictions = predict_func_2input_2out(
    model, data_test_reshaped, light_alpha_test, batch_size=1)
mean_predictions = mean_predictions * (target_max - target_min) + target_min
mean_predictions = mean_predictions.numpy()
sigma_predictions = sigma_predictions * (target_max - target_min)
sigma_predictions = sigma_predictions.numpy()

In [15]:
all_predictions = mean_predictions * whitelight_s_test[:, np.newaxis]
newsigma = sigma_predictions * whitelight_s_test[:, np.newaxis]

In [16]:
wavelengths = pd.read_csv(
    '/kaggle/input/ariel-data-challenge-2024/wavelengths.csv')
test_adc_info = pd.read_csv(
    '/kaggle/input/ariel-data-challenge-2024/test_adc_info.csv',
    index_col='planet_id')
train_labels = pd.read_csv(
    '/kaggle/input/ariel-data-challenge-2024/train_labels.csv',
    index_col='planet_id')

In [19]:
sub_df = postprocessing(all_predictions,
                        wavelengths,
                        test_adc_info.index,
                        sigma_pred=newsigma)

In [20]:
display(sub_df)

,wl_1,wl_2,wl_3,wl_4,wl_5,wl_6,wl_7,wl_8,wl_9,wl_10,...,sigma_274,sigma_275,sigma_276,sigma_277,sigma_278,sigma_279,sigma_280,sigma_281,sigma_282,sigma_283
planet_id,,,,,,,,,,,,,,,,,,,,,
785834,0.001279,0.001319,0.001318,0.001318,0.001328,0.001326,0.001322,0.001328,0.001324,0.001331,...,0.000030,0.000031,0.000030,0.000029,0.000032,0.000031,0.000032,0.000031,0.000031,0.000031
14485303,0.001799,0.001857,0.001855,0.001854,0.001868,0.001867,0.001860,0.001869,0.001863,0.001873,...,0.000042,0.000044,0.000042,0.000041,0.000044,0.000043,0.000045,0.000043,0.000043,0.000044
17002355,0.002646,0.002730,0.002727,0.002726,0.002746,0.002744,0.002734,0.002748,0.002739,0.002754,...,0.000062,0.000064,0.000062,0.000061,0.000065,0.000064,0.000066,0.000063,0.000064,0.000065
24135240,0.001367,0.001411,0.001410,0.001409,0.001420,0.001419,0.001413,0.001420,0.001416,0.001423,...,0.000032,0.000033,0.000032,0.000032,0.000034,0.000033,0.000034,0.000033,0.000033,0.000033
25070640,0.001936,0.001997,0.001995,0.001994,0.002009,0.002007,0.002000,0.002010,0.002003,0.002014,...,0.000045,0.000047,0.000045,0.000045,0.000048,0.000047,0.000048,0.000046,0.000047,0.000047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4249337798,0.000787,0.000812,0.000811,0.000811,0.000817,0.000816,0.000813,0.000817,0.000815,0.000819,...,0.000019,0.000019,0.000019,0.000018,0.000019,0.000019,0.000020,0.000019,0.000019,0.000019
4257395405,0.003881,0.004004,0.004000,0.003998,0.004027,0.004024,0.004010,0.004030,0.004016,0.004038,...,0.000091,0.000094,0.000091,0.000089,0.000096,0.000093,0.000097,0.000093,0.000093,0.000095
4266129805,0.006239,0.006437,0.006430,0.006428,0.006475,0.006469,0.006447,0.006479,0.006457,0.006493,...,0.000147,0.000152,0.000147,0.000144,0.000154,0.000150,0.000156,0.000149,0.000150,0.000152


In [21]:
sub_df.to_csv('/kaggle/working/submission.csv', index=True)